In [32]:
def counter(i):
    base = i
    def inc(x=1):
        nonlocal base
        base += x
        return base
    return inc

In [2]:
inc = counter(3)

In [3]:
inc(3)

6

返回函数或者参数是函数的函数 -- 高阶函数

因为Python中函数是一等对象(first class)

函数也是对象，并且它可以像普通对象一样赋值，作为参数，作为返回值

In [4]:
def fn():
    print('fn called')

In [5]:
f = fn

In [6]:
f()

fn called


In [26]:
def sort(it, cmp=lambda a, b: a < b):    
    ret = []
    for x in it:
        for i, e in enumerate(ret):
            if cmp(x, e):
                ret.insert(i, x)
                break
        else:
            ret.append(x)
    return ret
                

In [30]:
sort([1, 3, 2, 4, 6, 8, 5], lambda a, b: a>b)

[8, 6, 5, 4, 3, 2, 1]

函数作为返回值： 通常是用于闭包的场景， 需要封装一些变量

函数作为参数：通常用于大多数逻辑固定，少部分逻辑不固定的场景

In [39]:
import datetime

def logger(fn): # 函数作为返回值： 封装了fn
    def wrap(*args, **kwargs):
        start = datetime.datetime.now()
        ret = fn(*args, **kwargs)
        end = datetime.datetime.now()
        print('call {} took {}'.format(fn.__name__, end-start))
        return ret
    return wrap

In [34]:
def add(x, y):
    return x + y

In [40]:
loged_add = logger(add)

In [41]:
loged_add(3, y=5)

call add took 0:00:00.000030


8

函数作为参数，返回值也是函数： 通常用于作为参数函数执行前后需要一些额外操作

In [42]:
import time

def sleep(x):
    time.sleep(x)

In [43]:
logged_sleep = logger(sleep)

In [44]:
logged_sleep(3)

call sleep took 0:00:03.005698


In [45]:
sleep = logger(sleep)

In [46]:
sleep(3)

call sleep took 0:00:03.003675


In [47]:
@logger
def sleep(x):
    time.sleep(x)

In [48]:
sleep(3)

call sleep took 0:00:03.003233


装饰器

参数是**一个函数**， 返回值是**一个函数**的函数，就可以作为装饰器

In [49]:
help(list.insert)

Help on method_descriptor:

insert(...)
    L.insert(index, object) -- insert object before index



In [50]:
help(add)

Help on function add in module __main__:

add(x, y)



In [51]:
def fn():
    '''this is fn'''

In [52]:
help(fn)

Help on function fn in module __main__:

fn()
    this is fn



In [53]:
fn.__doc__

'this is fn'

In [54]:
fn.__name__

'fn'

In [56]:
sleep.__name__

'wrap'

In [57]:
import datetime

def logger(fn): # 函数作为返回值： 封装了fn
    def wrap(*args, **kwargs):
        start = datetime.datetime.now()
        ret = fn(*args, **kwargs)
        end = datetime.datetime.now()
        print('call {} took {}'.format(fn.__name__, end-start))
        return ret
    wrap.__name__ = fn.__name__
    wrap.__doc__ = fn.__doc__
    return wrap

In [62]:
@logger
def sleep(x):
    time.sleep(x)

In [63]:
sleep.__name__

'sleep'

In [64]:
def copy_proprities(src, dst):
    dst.__name__ =  src.__name__
    dst.__doc__ = src.__doc__

In [65]:
import datetime

def logger(fn): # 函数作为返回值： 封装了fn
    def wrap(*args, **kwargs):
        start = datetime.datetime.now()
        ret = fn(*args, **kwargs)
        end = datetime.datetime.now()
        print('call {} took {}'.format(fn.__name__, end-start))
        return ret
    copy_proprities(fn, wrap)
    return wrap

In [70]:
def copy_proprities(src):
    def _copy(dst):
        dst.__name__ =  src.__name__
        dst.__doc__ = src.__doc__
        return dst
    return _copy
        

In [71]:
import datetime

def logger(fn): # 函数作为返回值： 封装了fn
    @copy_proprities(fn)
    def wrap(*args, **kwargs):
        start = datetime.datetime.now()
        ret = fn(*args, **kwargs)
        end = datetime.datetime.now()
        print('call {} took {}'.format(fn.__name__, end-start))
        return ret
    return wrap

In [72]:
@logger
def sleep(x):
    time.sleep(x)

In [73]:
sleep.__name__

'sleep'

柯里化

In [74]:
import functools

In [77]:
import datetime

def logger(fn): # 函数作为返回值： 封装了fn
    @functools.wraps(fn)
    def wrap(*args, **kwargs):
        start = datetime.datetime.now()
        ret = fn(*args, **kwargs)
        end = datetime.datetime.now()
        print('call {} took {}'.format(fn.__name__, end-start))
        return ret
    return wrap

In [78]:
@logger
def sleep(x):
    time.sleep(x)

In [79]:
sleep.__name__

'sleep'

In [80]:
help(functools.wraps)

Help on function wraps in module functools:

wraps(wrapped, assigned=('__module__', '__name__', '__qualname__', '__doc__', '__annotations__'), updated=('__dict__',))
    Decorator factory to apply update_wrapper() to a wrapper function
    
    Returns a decorator that invokes update_wrapper() with the decorated
    function as the wrapper argument and the arguments to wraps() as the
    remaining arguments. Default arguments are as for update_wrapper().
    This is a convenience function to simplify applying partial() to
    update_wrapper().



In [81]:
s = datetime.datetime.now()

In [82]:
e = datetime.datetime.now()

In [84]:
delta = e - s

In [86]:
delta.total_seconds()

13.524782

In [87]:
def logger(s):
    def _logger(fn):
        @functools.wraps(fn)
        def wrap(*args, **kwargs):
            start = datetime.datetime.now()
            ret = fn(*args, **kwargs)
            end = datetime.datetime.now(ii)
            if (end-start).total_seconds() > s:
                print('call {} took {}'.format(fn.__name__, end-start))
            return ret
        return wrap
    return _logger
    

In [88]:
@logger(2)
def sleep(x):
    time.sleep(x)

In [89]:
sleep(3)

call sleep took 0:00:03.004279


In [90]:
sleep(1)

带参数的装饰器： 一个函数， 返回一个不带参数的装饰器

In [96]:
def logger(s, p=lambda name, t: print('call {} took {}'.format(name, t))):

    def _logger(fn):
        
        @functools.wraps(fn)
        def wrap(*args, **kwargs):
            
            start = datetime.datetime.now()
            ret = fn(*args, **kwargs)
            end = datetime.datetime.now()
            if (end-start).total_seconds() > s:
                p(fn.__name__, end-start)
            return ret
        return wrap
    return _logger
    

In [97]:
@logger(2, p=lambda name, t: None)
def sleep(x):
    time.sleep(x)

In [98]:
sleep(3)

In [100]:
def logger(s):
    def _logger(p=lambda name, t: print('call {} took {}'.format(name, t))):
        def __logger(fn):
            @functools.wraps(fn)
            def wrap(*args, **kwargs):
                start = datetime.datetime.now()
                ret = fn(*args, **kwargs)
                end = datetime.datetime.now()
                if (end-start).total_seconds() > s:
                    p(fn.__name__, end-start)
                return ret
            return wrap
        return __logger
    return _logger
    

In [104]:
@@logger(3)()
def sleep(x):
    time.sleep(x)

SyntaxError: invalid syntax (<ipython-input-104-62b2fa1c91f5>, line 1)

In [102]:
logger2s = logger(2)

In [103]:
@logger2s()
def sleep(x):
    time.sleep(x)

In [105]:
sleep(3)

call sleep took 0:00:03.003878


In [106]:
sleep(1)

In [ ]:
sleep = logger2s()(sleep)